In [13]:
# Importing the libraries
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np
import spacy

In [14]:
# Function to extract keywords from a single document
def extract_keywords(doc, top_n=3):
    tfidf = TfidfVectorizer(stop_words='english', max_features=top_n)
    response = tfidf.fit_transform([doc])
    feature_names = np.array(tfidf.get_feature_names_out())
    tfidf_scores = response.toarray()[0]
    top_indices = tfidf_scores.argsort()[-top_n:][::-1]
    return feature_names[top_indices]

In [15]:
# Example function to train an SVR model on bullet points with importance scores
def train_svr_model(bullet_points, scores):
    tfidf = TfidfVectorizer(stop_words='english')
    X = tfidf.fit_transform(bullet_points)
    y = np.array(scores)

    svr = make_pipeline(StandardScaler(with_mean=False), SVR(C=1.0, epsilon=0.2))
    svr.fit(X, y)

    return tfidf, svr

In [16]:
# Example bullet points and corresponding importance scores
bullet_points = [
    "Math is all around us.", 
    "Math helps us understand the world and solve problems.", 
    "This text will explain what math is, why it is important, and how we use it in our daily lives.", 
    "There are many different areas of math, such as addition, subtraction, multiplication, division, geometry, and algebra.", 
    "Another important part of math is geometry.", 
    "Algebra is another area of math that is very important.", 
    "Math is important for many reasons.", 
    "When we use math, we learn to think logically and critically.", 
    "Second, math is important for many jobs.", 
    "Even jobs that do not seem related to math often require basic math skills.", 
    "For example, chefs use math to measure ingredients, and cashiers use math to give the correct change.", 
    "Third, math helps us understand the world.", 
    "We also use math in our daily lives in many ways.", 
    "For example, when we cook, we use math to measure ingredients and follow recipes.", 
    "In conclusion, math is a very important subject that helps us in many ways.", 
    "We use math every day, often without even thinking about it.", 
    "Math is a valuable tool that helps us in many areas of our lives."
]
print(len(bullet_points))
importance_scores = [8.5, 9.0, 7.5, 8.2, 7.8, 9.1, 8.9, 7.6, 8.0, 9.3, 7.7, 8.1, 8.4, 9.2, 7.9, 8.3, 8.7]

# Train the SVR model
tfidf, svr = train_svr_model(bullet_points, importance_scores)

17


In [17]:
def generate_slide_deck_title(bullet_points, top_n=3):
    # Combine all bullet points into a single document
    combined_text = ' '.join(bullet_points)

    nlp = spacy.load('en_core_web_lg')
    doc = nlp(combined_text)
    
    # Extract key phrases using spaCy noun chunks
    key_phrases = [chunk.text for chunk in doc.noun_chunks]
    
    # Predict importance scores for key phrases using the SVR model
    phrase_scores = []
    for phrase in key_phrases:
        X = tfidf.transform([phrase])
        score = svr.predict(X)
        phrase_scores.append((phrase, score[0]))
    
    # Sort phrases by their predicted importance scores
    sorted_phrases = sorted(phrase_scores, key=lambda x: x[1], reverse=True)
    
    # Get the top N phrases
    top_phrases = [phrase for phrase, score in sorted_phrases[:top_n]]
    
    # Capitalize each word in each phrase
    capitalized_phrases = [' '.join(word.capitalize() for word in phrase.split()) for phrase in top_phrases]
    
    # Generate a title by joining the top N key phrases
    title = ' '.join(capitalized_phrases)
    
    return title

In [18]:
# Generate a slide deck title
title = generate_slide_deck_title(bullet_points, top_n=3)
print(title)

Many Reasons Another Area Algebra
